The purpose of this script is to create 2 dataframes: 'sentence_topic_judgeid.csv' and 'judge_decision.csv'
1. **sentence_topic_judgeid**: full opinion text, author judge's full name and id, with the topic and geniss code of each case. 
2. **judge_decision**: judge level decision (binary: affirm(1), reverse(0)) of each judge toward each case.

## Data Description
1. $266568$ circuit opinion writings. 
2. $2099$ judges
3. $89$ unique topics in the dataset.

In [2]:
import pandas as pd
import numpy as np
import pickle as pk
import zipfile
import json
import string
import os


In [3]:
#original dataset
sentence_path = '/data/Dropbox/judge_embedding_data_sp18/sentences_data.csv'
ruling_path = '/data/Dropbox/Projects/originalism/data/BloombergVOTELEVEL_Touse.dta'
district_path = '/data/Dropbox/Projects/Ash_Chen/lexis-data/6411.zip'
topic_path = '/data/Dropbox/Projects/Ash_Chen/metadata/bb2topic.pkl'
meta_path = '/data/Dropbox/judge_embedding_data_sp18/circuit_metadata_excerpt.dta'
bloomberg_lexis_path = '/data/Dropbox/judge_embedding_data_sp18/caseid_citation.dta'

#output directory
data_output_path = '/data/Dropbox/judge_embedding_data_sp18/'

#preprocessed dataset
sentence_topic_judge_path = data_output_path+ 'sentence_topic_judgeid.csv'
judge_decision_path = data_output_path+ 'judge_decision.csv'
circuit_output_path = data_output_path+ 'circuit/' 
blmberg_lexis_circuit_path = data_output_path+ 'blmberg_lexis_circuit.csv'
circuit_district_link_path = data_output_path + 'circuit_district_link.csv'

In [4]:
import psutil
def show_current_memory_usage():
    pid = os.getpid()
    py = psutil.Process(pid)
    mem_use_in_GB = py.memory_info().rss/(2**30)
    print("currently using",mem_use_in_GB,"GB memory!")

## Sentence Data

In [4]:
df_sentence=pd.read_csv(sentence_path, index_col=0)

In [5]:
df_sentence.head()

,caseid,year,judge_last_name,middle_part,opinion_text
0,XB0PMNQNB5G0,1891,PARDEE,contentMajOp,"P ARDEE , J., ( after stating the case .)\nTh..."
1,XFL757,1891,BROWN,contentMajOp,"B ROWN , Circuit Justice, ( orally .)\nIn thi..."
2,XB0PMRQNB5G0,1891,PARDEE,contentMajOp,"P ARDEE , J. The case on this intervention is..."
3,XFL7KI,1891,LOCKE,contentMajOp,"L OCKE , District Judge.\nThis motion coming ..."
4,XB0OI7QNB5G0,1891,NELSON,contentMajOp,"N ELSON , J. This was an action brought to re..."


## Geniss Data

In [6]:
geniss = pk.load(open('/data/Dropbox/Projects/Ash_Chen/metadata/bb2genis.pkl', 'rb') )
df_geniss = pd.DataFrame(geniss, index=[0]).transpose()
df_geniss.reset_index(level=0, inplace = True)
df_geniss.columns = ['caseid', 'geniss']

In [7]:
df_geniss.head()

,caseid,geniss
0,X101L14003,4.0
1,X10284I003,1.0
2,X102UPO003,4.0
3,X103660N,9.0
4,X1036DGN,6.0


## Topic

In [8]:
topic = pk.load(open(topic_path, 'rb') )
df_topic = pd.DataFrame(topic, index=[0]).transpose()
df_topic.reset_index(level=0, inplace = True)
df_topic.columns = ['caseid', 'topic']


In [9]:
df_topic.head()

,caseid,topic
0,X101L14003,Civil Procedure
1,X10284I003,Criminal Law
2,X102UPO003,Appellate Procedure
3,X103660N,Transportation Law
4,X1036DGN,Employment Law


## Ruling data

In [14]:
df_ruling_key = pd.read_stata(ruling_path,
                              columns = ['songername', 'judgeidentificationnumber', 
                                         'jOrigname', 'caseid'])

In [97]:
# df_judge_bio = df_ruling[['songername', 'judgeidentificationnumber', 'jOrigname', 'caseid','party', 'x_dem', 'x_republican','x_protestant',
#  'x_evangelical','x_noreligion','x_catholic','x_jewish', 'x_black', 'x_nonwhite', 'x_female',
# 'nameofschool1','nameofschool2','nameofschool3','nameofschool4', 'nameofschool5','Circuitjudge']]

## Merge: create 'sentence_topic_judge.csv'

In [31]:
#1 Sentence + ruling to get judge id and date of case 
df_merged = df_sentence.merge(df_ruling_key, left_on=['caseid', 'judge_last_name'], 
                              right_on=['caseid', 'jOrigname'], 
                              how = 'inner')
df_merged.drop(['judge_last_name', 'jOrigname'], axis = 1, inplace = True)


#2 + Topic
df_merged = df_merged.merge(df_topic, on = 'caseid', how = 'left')

#3 + Geniss
df_merged = df_merged.merge(df_geniss, on = 'caseid', how = 'left')

#drop where songername is null (no matching in ruling data)
df_merged.dropna(axis=0, subset = ['songername'], inplace = True)

df_merged.head()

,caseid,year,middle_part,opinion_text,songername,judgeidentificationnumber,topic,geniss
0,XB0PMNQNB5G0,1891,contentMajOp,"P ARDEE , J., ( after stating the case .)\nTh...","PARDEE, DON A.",1830.0,Civil Procedure,4.0
1,XFL757,1891,contentMajOp,"B ROWN , Circuit Justice, ( orally .)\nIn thi...","BROWN, HENRY BILLINGS",284.0,Civil Procedure,4.0
2,XB0PMRQNB5G0,1891,contentMajOp,"P ARDEE , J. The case on this intervention is...","PARDEE, DON A.",1830.0,Contracts,7.0
3,XFL7KI,1891,contentMajOp,"L OCKE , District Judge.\nThis motion coming ...","LOCKE, JAMES WILLIAM",1410.0,Civil Procedure,4.0
4,XB0OI7QNB5G0,1891,contentMajOp,"N ELSON , J. This was an action brought to re...","NELSON, SUSAN RICHARD",3339.0,NaN,NaN


In [12]:
print('There are {0} unique judges in the dataset.'.format(df_merged.songername.nunique()))
print('There are {0} unique cases in the dataset. '.format(df_merged.caseid.nunique()))
print('There are {0} unique topics in the dataset. (Excluding Nan) '.format(df_merged.topic.nunique()))

# print('Unique cases by year', df_merged.groupby('year').caseid.nunique())
# print('Unique writings by judge', '\n', df_merged.groupby('songername').caseid.count())

There are 2099 unique judges in the dataset.
There are 266568 unique cases in the dataset. 
There are 89 unique topics in the dataset. (Excluding Nan) 


In [52]:
# df_merged.to_csv(sentence_topic_judge_path, index = False)

## Judge Level Decision: create 'judge_decision.csv'

In [4]:
df_meta = pd.read_stata(meta_path)
df_meta.shape

(1163694, 15)

In [5]:
df_meta.head()

,caseid,Circuit,j,date,songername,Affirmed,AffirmedInPart,Reversed,ReversedInPart,Vacated,VacatedInPart,Remanded,Dissenting1,Author,Writer
0,X101L14003,4,2,2004-03-24,"DUNCAN, ALLYSON KAY",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,MOTZ,1.0
1,X101L14003,4,3,2004-03-24,"KING, ROBERT BRUCE",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,MOTZ,1.0
2,X101L14003,4,1,2004-03-24,"MOTZ, DIANA GRIBBON",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,MOTZ,1.0
3,X10284I003,4,3,2004-03-31,"LUTTIG, J. MICHAEL",0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,WILKINSON,1.0
4,X10284I003,4,1,2004-03-31,"WILKINSON, J. HARVIE, III",0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,WILKINSON,1.0


In [7]:
df_ruling_affirm = pd.read_stata(ruling_path, columns = ['caseid', 'affirmdummy'])
df_ruling_affirm.drop_duplicates(inplace = True)

In [8]:
df_ruling_affirm.affirmdummy.fillna(-1, inplace = True)
df_ruling_affirm.groupby('affirmdummy').caseid.nunique()/df_ruling_affirm.shape[0]

affirmdummy
-1.0    0.000018
 0.0    0.439386
 1.0    0.560596
Name: caseid, dtype: float64

In [9]:
df_merged = pd.read_csv(sentence_merge_path)

In [10]:
df_meta = df_meta[df_meta['caseid'].isin(df_merged.caseid)]
df_meta.shape

(799704, 15)

In [11]:
df_meta = df_meta.merge(df_ruling_affirm, on='caseid', how= 'left')
df_meta.shape

(799704, 16)

In [14]:
df_meta.groupby(['affirmdummy']).caseid.nunique()/df_meta.caseid.nunique()

affirmdummy
0.0    0.452597
1.0    0.547403
Name: caseid, dtype: float64

In [15]:
df_meta['dissent'] = df_meta.apply(lambda x: 1 if x['j']==x['Dissenting1'] else 0, axis = 1)

In [28]:
def decision(x):
    if x['affirmdummy'] == 1: #affirm
        if x['dissent'] == 0:
            return 1 #affirm
        elif x['dissent'] == 1:
            return 0 #reverse
    elif x['affirmdummy'] == 0: #reverse
        if x['dissent'] == 0:
            return 0 #reverse
        elif x['dissent'] == 1:
            return 1

df_meta['judge_decision'] = df_meta.apply(decision, axis = 1)

In [31]:
df_meta = df_meta[['caseid','Circuit','date','songername','affirmdummy','judge_decision']]

In [34]:
df_meta.groupby('judge_decision').caseid.count()/df_meta.shape[0]

judge_decision
0    0.447746
1    0.552254
Name: caseid, dtype: float64

In [9]:
df_meta.head()

,caseid,Circuit,date,songername,affirmdummy,judge_decision
0,X10284I003,4,2004-03-31,"LUTTIG, J. MICHAEL",0.0,0
1,X10284I003,4,2004-03-31,"WILKINSON, J. HARVIE, III",0.0,0
2,X10284I003,4,2004-03-31,"TRAXLER, WILLIAM BYRD, JR.",0.0,0
3,X102UPO003,3,2004-04-05,"RENDELL, MARJORIE O.",0.0,0
4,X102UPO003,3,2004-04-05,"SLOVITER, DOLORES K.",0.0,0


In [ ]:
# df_meta.to_csv(judge_decision_path, index = False)

## district id extraction
#### Please refer to district_id.ipynb for the script to create '/data/Dropbox/judge_embedding_data_sp18/district_opinion_list.pk'

## circuit and district ID link extraction
#### Please refer to district_id.ipynb for the script to create '/data/Dropbox/judge_embedding_data_sp18/circuit_district_XXXX.pk'

## bloomberg_lexis link

In [5]:
blmberg_lexis_df = pd.read_stata(bloomberg_lexis_path)

In [6]:
blmberg_lexis_df.head()

,caseid,citation
0,XFLM55,112 F.1d 517
1,XFKIHI,92 F.1d 525
2,XFJ66N,100 F.1d 34
3,XFKKU1,109 F.1d 334
4,XFKHRR,137 F.1d 8


In [7]:
all_blmberg_caseid = pd.read_csv(judge_decision_path, usecols = ['caseid', 'Circuit'])

In [8]:
blmberg_lexis_circuit = blmberg_lexis_df.merge(all_blmberg_caseid, on = 'caseid', how = 'right')
blmberg_lexis_circuit.drop_duplicates(inplace = True)
blmberg_lexis_circuit.dropna(axis=0, how='any', inplace = True)
# blmberg_lexis_circuit.to_csv(blmberg_lexis_circuit_path, index = False)

In [9]:
blmberg_lexis_circuit = pd.read_csv(data_output_path+'blmberg_lexis_circuit.csv')
col_names =  ['circuit_id', 'district_id', 'caseid', 'Circuit']
circuit_district_df = pd.DataFrame(columns = col_names)

circuit_district_file_list = ['6385', '6386', '6387', '6388', '6389', '6390', 
                             '6391', '6392', '6393', '6394', '6395', '6396', '6397']
for i in circuit_district_file_list:
    try:
        circuit_district = pk.load(open( data_output_path + "circuit_district_{0}.pk".format(i), "rb" ) )
    except:
        pass
    else:
        circuit_district = circuit_district[pd.notnull(circuit_district['circuit_id'])]
        circuit_district_sub = circuit_district[['circuit_id', 'district_id']]
        circuit_district_sub = circuit_district_sub.drop_duplicates()
        circuit_district_new = circuit_district_sub.merge(blmberg_lexis_circuit, 
                                                           left_on=['circuit_id'], right_on=['citation'], 
                                                           how = 'right')
        circuit_district_new = circuit_district_new[pd.notnull(circuit_district_new['circuit_id'])]
        circuit_district_new = circuit_district_new[['circuit_id', 'district_id', 'caseid', 'Circuit']]
        circuit_district_df = pd.concat([circuit_district_df, circuit_district_new])



In [10]:
circuit_district_df.to_csv(circuit_district_link_path, index = False)

In [11]:
circuit_district_df.head()

,circuit_id,district_id,caseid,Circuit
0,166 F.3d 6,NaN,X37VS7,1
1,174 F.3d 36,NaN,X36A8T,1
2,175 F.3d 37,NaN,X36I8R,1
3,178 F.3d 572,NaN,XACCU3,1
4,180 F.3d 349,NaN,X4VQR3,1


In [12]:
circuit_district_df['district_id'].notnull().sum()

8386

In [15]:
sorted(circuit_district_df['Circuit'].unique())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]